#  Install depences

In [ ]:
!apt-get update -y
#!apt-get upgrade -y
!apt install libgl1-mesa-glx -y
!pip install matplotlib
!pip install opencv-python
!/usr/bin/python -m pip install --upgrade pip
print('Depencies Installed')

# Import Libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import PIL
import pathlib
import os
import datetime
import urllib.request
import cv2 
from random import seed
from random import randint
import tensorflow as tf
import logging


print('Librariew Imported')

# Initialize Vars

In [ ]:

AUTOTUNE = tf.data.AUTOTUNE

BATCH_SIZE = 32
WIDTH=240
HEIGHT=240
IMG_SIZE = (WIDTH, HEIGHT)
filename="Count_insects_final_2"

project_folder=""
datadir_local=pathlib.Path('images_sets')

dataset_folder_local=os.path.join(datadir_local,"images_train_set")


check_point_folder=os.path.join(project_folder,'check_point_folder')
!mkdir {check_point_folder}

log_folder_path=os.path.join(project_folder,'logs')
!mkdir {log_folder_path}

history_log_folder_path=os.path.join(project_folder,'history_logs')
!mkdir {history_log_folder_path}

saved_folder_path=os.path.join(project_folder,'saved_models')
!mkdir {saved_folder_path}
print('Initialize Vars')



In [ ]:
try:
    for h in logging.getLogger().handlers:
        logging.getLogger().removeHandler(h)
except:
    pass
mylogs = logging.getLogger('CNN_Count_insects_final_2')
mylogs.setLevel(logging.INFO)
# Handler - 1
h_file = logging.FileHandler(f'{filename}.log')
fileformat = logging.Formatter("%(asctime)s:%(levelname)s:%(message)s")
h_file.setLevel(logging.INFO)
h_file.setFormatter(fileformat)

h_stream = logging.StreamHandler()
h_streamformat = logging.Formatter("%(asctime)s:   %(message)s")
h_stream.setLevel(logging.INFO)
h_stream.setFormatter(h_streamformat)

# Adding all handlers to the logs
mylogs.addHandler(h_file)
mylogs.addHandler(h_stream)
mylogs.info('Initialize Log')


# Create TF Dataset

In [ ]:
imagePaths= []
dataset_file_list=os.path.join(datadir_local,'list_insects_14000.txt')
if not os.path.exists(dataset_file_list):
  ###
  #**********************
  DATASET_SIZE=0
  for imagefiles in os.listdir(dataset_folder_local):
    if imagefiles[-3:]!='txt':
      imagePaths.append(os.path.join(dataset_folder_local,imagefiles))
      DATASET_SIZE=DATASET_SIZE+1
  np.random.shuffle(imagePaths)
  with open(dataset_file_list,'w') as f:
    f.write(str(imagePaths))
  #*********************
  ###
else:
  with open(dataset_file_list,'r') as f:
    imagePaths=eval(f.readline())
  DATASET_SIZE=len(imagePaths)
  

def load_images(imagePath):
  image = tf.io.read_file(imagePath)
  image = tf.image.decode_jpeg(image, channels=3)
  image = tf.image.rgb_to_grayscale(image)
  #image = tf.image.resize(image, (240, 240)) / 255.0
  label=int(tf.strings.split(tf.strings.split(imagePath,'_')[-1],'.')[-2])
  return (image, label)

mylogs.info(len(imagePaths))

train_imagePaths = imagePaths[:int(len(imagePaths)*0.7)]
mylogs.info(len(train_imagePaths))

train_ds = tf.data.Dataset.from_tensor_slices(train_imagePaths).map(load_images, num_parallel_calls=AUTOTUNE)
train_ds = train_ds.cache()
#train_ds = train_ds.repeat()
train_ds = train_ds.batch(BATCH_SIZE)
train_ds = train_ds.prefetch(AUTOTUNE)
mylogs.info('train_ds ok')

test_imagePaths = imagePaths[int(len(imagePaths)*0.7):]
mylogs.info(len(test_imagePaths))

val_ds = tf.data.Dataset.from_tensor_slices(test_imagePaths).map(load_images, num_parallel_calls=AUTOTUNE)
val_ds = val_ds.cache()
#val_ds = val_ds.repeat()
val_ds = val_ds.batch(BATCH_SIZE)
val_ds = val_ds.prefetch(AUTOTUNE)
mylogs.info('val_ds ok')
mylogs.info(f'Data set folder {dataset_folder_local} \n Train set size {len(train_ds)}  \n Validation set size {len(val_ds)}  \n General size {DATASET_SIZE}')



# Show sample of dataset

In [ ]:
plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
  for i in range(12):
    ax = plt.subplot(3, 4, i + 1)
    plt.imshow(images[i][:,:,0],cmap='gray', vmin=0, vmax=255)
    plt.title(str(labels[i].numpy()))
    plt.axis("off")

# Augmentation image dataset

In [ ]:
data_augmentation = tf.keras.Sequential([
  #tf.keras.layers.experimental.preprocessing.Resizing(WIDTH, HEIGHT),
  tf.keras.layers.experimental.preprocessing.Rescaling(1./255),
  #tf.keras.layers.experimental.preprocessing.Rescaling(1./127.5, offset= -1),
  tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
  tf.keras.layers.experimental.preprocessing.RandomRotation(0.5),
  tf.keras.layers.experimental.preprocessing.RandomZoom(0.2)
])
data_augmentation2 = tf.keras.Sequential([
  #tf.keras.layers.experimental.preprocessing.Resizing(WIDTH, HEIGHT),
  tf.keras.layers.experimental.preprocessing.Rescaling(1./255),
  #tf.keras.layers.experimental.preprocessing.Rescaling(1./127.5, offset= -1),
  #tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
  #tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
  #tf.keras.layers.experimental.preprocessing.RandomZoom(0.1)
])

train_ds_augmentation=train_ds.map(lambda image, label: (data_augmentation(image, training=True), label),  num_parallel_calls=AUTOTUNE)
val_ds_augmentation=val_ds.map(lambda image, label: (data_augmentation(image, training=True), label), num_parallel_calls=AUTOTUNE)
mylogs.info(f'train_ds_augmentation {len(train_ds_augmentation)}    val_ds_augmentation {len(val_ds_augmentation)} ')
mylogs.info('Augmention Complete')


# Create  CNN

In [ ]:

opt=tf.keras.optimizers.Adam( learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, amsgrad=False)
CNN_layers=[
    {"filters":16, "kernel":5,"kernel_regulazire":False,"batchnormalization":False,"poolmax":True},
    {"filters":16, "kernel":5,"kernel_regulazire":False,"batchnormalization":False,"poolmax":True},
    {"filters":32, "kernel":3,"kernel_regulazire":False,"batchnormalization":False,"poolmax":True},
    {"filters":32, "kernel":3,"kernel_regulazire":False,"batchnormalization":False,"poolmax":True},
]
    

input_tensor = tf.keras.Input(shape=(WIDTH,HEIGHT,1))
for i,CNN_layer in enumerate(CNN_layers):
    
    if i==0:
        CNN=input_tensor
    
    CNN=tf.keras.layers.Conv2D(filters=CNN_layer['filters'] ,
                              kernel_size=CNN_layer['kernel'], 
                              padding='same', 
                              activation="relu",
                              kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.01),
                              bias_initializer=tf.keras.initializers.Zeros()
                              #, kernel_regularizer=tf.keras.regularizers.l2(0.01)
                              )(CNN)  

    if CNN_layer['batchnormalization']==True:
        CNN=tf.keras.layers.BatchNormalization(axis=-1)(CNN)

    if  CNN_layer['poolmax']==True:
        CNN = tf.keras.layers.MaxPool2D(pool_size=(2, 2),padding='same')(CNN)

CNN=tf.keras.layers.Flatten(name="FLATTEN")(CNN)
CNN= tf.keras.layers.Dense(64 ,name='FC_1',activation="relu")(CNN)
CNN= tf.keras.layers.Dense(32 ,name='FC_2',activation="relu")(CNN)
CNN= tf.keras.layers.Dense(1 ,name='FC_3',activation="linear")(CNN)

model= tf.keras.Model(inputs=input_tensor, outputs=CNN,name='model_count_insects_final_2' )
model.compile(optimizer=opt,
              loss=tf.keras.losses.MeanSquaredError(),
              metrics=['mae'])

mylogs.info(model.summary())
path_saved_model=os.path.join(saved_folder_path,f"{model.name}.h5")

## Evalute model

In [ ]:
val_ds_a=val_ds.map(lambda image,label:(image/255,label),num_parallel_calls=AUTOTUNE)
loss0,mae = model.evaluate(val_ds_a,verbose=2)


# Train model

## Prepare Train

In [ ]:

%load_ext tensorboard

initial_epochs = 250

logdir = os.path.join('logs', f'{datetime.datetime.now().strftime("%Y%m%d-%H%M%S")}')
mylogs.info(f'logdir {logdir}')


checkpoint_folder_path=os.path.join(check_point_folder,f'{model.name}')

!mkdir {checkpoint_folder_path}

checkpoint_filepath=os.path.join(checkpoint_folder_path,'weights-improvement-{epoch:02d}-{val_loss:.2f}.hdf5')


model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True
    )

history_log_file=os.path.join(history_log_folder_path,f'{model.name}-log.csv')
history_logger_callback=tf.keras.callbacks.CSVLogger(history_log_file, separator=",", append=True)

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir, histogram_freq=1)

callbacks=[tensorboard_callback,model_checkpoint_callback,history_logger_callback]
mylogs.info('pretraining ok')


## Train Model

In [ ]:

mylogs.info(f'{model.name}  Start {datetime.datetime.now().strftime("%Y%m%d-%H%M%S")}')

%tensorboard --logdir logs

  
model.fit(train_ds_augmentation,
                      epochs=initial_epochs,
                      validation_data=val_ds_augmentation,
                      batch_size=BATCH_SIZE,
                      callbacks=callbacks,
                      verbose=2
                      )


model.save(path_saved_model)
mylogs.info(f' Model saved in {path_saved_model}')


## Save trained model

In [ ]:
path_saved_model=os.path.join(saved_folder_path,f"{model.name}.h5")
model.save(path_saved_model)
print(f' Model saved in {path_saved_model}')

## Evaluate model 2

In [ ]:
model_ev = tf.keras.models.load_model(path_saved_model)
val_ds_a=val_ds.map(lambda image,label:(image/255,label),num_parallel_calls=AUTOTUNE)
loss0,mae = model_ev.evaluate(val_ds_a,verbose=2)

## Accure model

In [ ]:

#check_point_file=os.path.join(check_point_folder,'{model.name}-weights-improvement-61-0.16.hdf5')
#print(check_point_file)
#model.load_weights(check_point_file)
#model = tf.keras.models.load_model(path_saved_model)

batch_prediction_floor = []
batch_prediction_round = []
batch_truth = []
count=0

for image, label in val_ds.unbatch():
    batch_truth.append(label)

    input_data =(np.float32(image)/255)
    input_data = np.expand_dims(input_data, axis=0)
    #print(input_data.shape)
    output_data=model.predict(input_data)

    #print(f'output_data {output_data}')
    predictions=np.floor(np.array(output_data).item(0))
    predictions_round=np.around(np.array(output_data).item(0))
    #print(f'predictions {predictions}')
    count=count+1
    batch_prediction_floor.append(predictions)
    batch_prediction_round.append(predictions_round)
mylogs.info(f'Number of test Images {count}')
tflite_accuracy = tf.keras.metrics.Accuracy()
tflite_accuracy(batch_prediction_floor, batch_truth)
mylogs.info("TF Lite accuracy: {:.3%}".format(tflite_accuracy.result()))


tflite_accuracy_round = tf.keras.metrics.Accuracy()
tflite_accuracy_round(batch_prediction_round, batch_truth)
mylogs.info("TF Lite accuracy round : {:.3%}".format(tflite_accuracy_round.result()))



# Test model

In [ ]:
modelpredict = tf.keras.models.load_model(path_saved_model)
print(f'Load model {path_saved_model}')
plt.figure(figsize=(10, 20))
for images, labels in val_ds.shuffle(64).take(1):
  for i in range(20):
    
    plt.imshow(images[i][:,:,0],cmap='gray', vmin=0, vmax=255)
    plt.show()

    input_data =(np.float32(images[i])/255)
    input_data = np.expand_dims(input_data, axis=0)
    pred=modelpredict.predict(input_data)
    predictions_floor=np.floor(np.array(pred).item(0))
    predictions_round=np.around(np.array(pred).item(0))
    print(f'predict={pred}  floor{predictions_floor}  round{predictions_round} label= {labels[i]}')
    